In [1]:
import pandas as pd

In [2]:
ticker = "NFLX"

In [3]:
def read_annual_data(csv_file, ticker):
    data = pd.read_csv(csv_file, index_col=0)
    data = data[ticker]
    data = data.to_frame()
    data['%-chg'] = data.pct_change()
    data = data.loc[2009:2019]
    return data

In [4]:
def read_quarterly_data(csv_file, ticker):
    data = pd.read_csv(csv_file, index_col=0, parse_dates=True)
    data = data[ticker]
    data = data.resample('Y', convention='end').agg('mean')
    data = data.to_frame()
    data['%-chg'] = data[ticker].pct_change()
    data = data.loc['2009-01-01':'2019-12-31']
    data.index = data.index.strftime("%Y")
    return data

In [5]:
eps = read_annual_data("50-eps.csv", ticker)
fcf = read_annual_data("50-fcf.csv", ticker)
rev = read_annual_data("50-rev.csv", ticker)
bvps = read_quarterly_data("50-bvps.csv", ticker)

In [6]:
def add_annual_data_2col(writer, df, sheet_name):
    df.to_excel(writer, sheet_name=sheet_name)
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]

    # Create a new chart object.
    chart = workbook.add_chart({'type': 'line'})

    # Add a series to the chart.
    chart.add_series({
        'name': sheet_name,
        'categories': [sheet_name, 1, 0, 11, 0],
        'values': [sheet_name, 1, 1, 11, 1],
    })
    
    # Create a new column chart.
    column_chart = workbook.add_chart({'type': 'column'})
    
    # Add a series to the chart.
    column_chart.add_series({
        'name': "%-chg",
        'categories': [sheet_name, 1, 0, 11, 0],
        'values': [sheet_name, 1, 2, 11, 2],
        'y2_axis':    True,
    })

    chart.combine(column_chart)
    chart.set_title({'name': sheet_name})
    chart.set_x_axis({'name': 'Date'})
    chart.set_y_axis({'name': 'Value'})
    column_chart.set_y2_axis({'name': '%-chg'})
    
    # To set the labels on x axis not on 0
    chart.set_x_axis({
        'label_position': 'low',
        'num_font':  {'rotation': 45}
    })


    # Insert the chart into the worksheet.
    worksheet.insert_chart('E1', chart)

In [7]:
writer = pd.ExcelWriter('data4.xlsx', engine='xlsxwriter')

add_annual_data_2col(writer, rev, 'Revenue')
add_annual_data_2col(writer, eps, 'EPS')
add_annual_data_2col(writer, fcf, 'FCF')
add_annual_data_2col(writer, bvps, 'BVPS')

writer.close()

It shows in FCF that Netflix has no FREE cash flow

A company with negative free cash flow indicates an inability to generate enough cash to support the business. Free cash flow tracks the cash a company has left over after meeting its operating expenses.